# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "LDL Cholesterol Levels"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Endometrioid_Cancer_(UCEC)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.UCEC.sampleMap_UCEC_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.UCEC.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['CDE_ID_3226963',
 '_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_PANCAN',
 '_PANCAN_DNAMethyl_UCEC',
 '_PANCAN_RPPA_PANCAN_K8',
 '_PANCAN_UNC_RNAseq_PANCAN_K16',
 '_PANCAN_miRNA_PANCAN',
 '_PANCAN_mirna_UCEC']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['CDE_ID_3226963', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_DNAMethyl_UCEC', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_UCEC', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_treatment_co

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_last_known_alive', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [59.0, 54.0, 69.0, 51.0, 67.0],
 'days_to_birth': [nan, -19818.0, -25518.0, -18785.0, -24477.0],
 'days_to_collection': [957.0, 90.0, 811.0, 103.0, 5015.0],
 'days_to_death': [nan, nan, nan, nan, 3251.0],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [1249.0, 13.0, 1202.0, 313.0, nan],
 'days_to_last_known_alive': [nan, nan, nan, nan, nan],
 'days_to_new_tumor_event_additional_surgery_procedure': [nan,
  nan,
  517.0,
  nan,
  nan],
 'days_to_new_tumor_event_after_initial_treatment': [396.0,
  nan,
  517.0,
  nan,
  nan],
 'year_of_initial_pathologic_diagnosis': [2011.0,
  2013.0,
  2011.0,
  2013.0,
  1996.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Endometrioid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-2E-A9G8-01,1,59.0,0.0,-1.175592,0.864474,-0.531035,0.426928,-0.601278,-1.67421,0.080406,...,2.149614,-0.086682,-0.748878,-0.987117,-0.262433,4.06601,0.333923,-1.501827,0.998505,-2.315833
TCGA-4E-A92E-01,1,54.0,0.0,0.215608,-0.178726,7.416765,0.115128,-0.378178,-0.71591,1.207206,...,-2.090786,-0.086682,-0.207478,-1.587117,-0.143633,0.93371,0.174223,-0.877027,0.217905,0.162267
TCGA-5B-A90C-01,1,69.0,0.0,-0.764492,-2.620626,-0.531035,-0.360672,-0.411278,-0.38501,0.412606,...,-1.380786,-0.086682,1.194822,-0.407417,-1.269133,0.77641,0.573223,-0.710827,-0.555895,-0.691533
TCGA-5S-A9Q8-01,1,51.0,0.0,-0.907792,-3.403426,-0.531035,-0.019372,-1.538578,-1.41421,0.358106,...,-0.637486,1.357518,-0.748878,-1.587117,-1.495533,1.98251,0.406423,-1.374527,0.759805,-1.819733
TCGA-A5-A1OH-01,1,86.0,0.0,-1.823192,5.243674,-0.531035,-0.365172,-0.476078,-0.72361,0.165706,...,6.647914,-0.086682,0.330522,0.758183,0.961667,3.00791,0.208623,0.311373,-1.538495,-0.985533


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 187 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Endometrioid Cancer', the least common label is '0' with 13 occurrences. This represents 6.95% of the dataset.
The distribution of the feature 'Endometrioid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 65.0
  75%: 73.0
Min: 33.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 187 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: /content/drive/MyDrive/AI4Science_Public/output1/Shuyi/Endometrioid-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "LDL-Cholesterol-Levels"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE28893',
 'GSE34945',
 'GSE25935',
 'GSE111567',
 'GSE26105',
 'GSE27645',
 'GSE181339']

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE181339"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/content/drive/MyDrive/AI4Science_Public/data/GEO/LDL-Cholesterol-Levels/GSE181339/GSE181339_family.soft.gz',
 '/content/drive/MyDrive/AI4Science_Public/data/GEO/LDL-Cholesterol-Levels/GSE181339/GSE181339_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Study of the usefulness of human peripheral blood mononuclear cells for the analysis of metabolic recovery after weight loss (METAHEALTH-TEST)"
!Series_summary	"The aim of this study is to design and validate a test, METAHEALTH-TEST, based on gene expression analysis in blood cells, to quickly and easily analyse metabolic health. This test will be used to analyse metabolic improvement in overweight/obese individuals and in metabolically obese normal-weight (MONW) individuals after undergoing a weight loss intervention and/or an intervention for improvement in eating habits and lifestyle. Obesity and its medical complications are a serious health problem today. Using peripheral blood mononuclear cells (PBMC) as an easily obtainable source of transcriptomic biomarkers would allow to deepen into the knowledge of adaptations in response to increased adiposity that occur in internal homeostatic tissues, without the need of using invasive biopsies. Moreover, if PBMC were able 

In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM5494930,GSM5494931,GSM5494932,GSM5494933,GSM5494934,GSM5494935,GSM5494936,GSM5494937,GSM5494938,...,GSM5494998,GSM5494999,GSM5495000,GSM5495001,GSM5495002,GSM5495003,GSM5495004,GSM5495005,GSM5495006,GSM5495007
0,!Sample_characteristics_ch1,gender: Man,gender: Man,gender: Woman,gender: Woman,gender: Woman,gender: Man,gender: Woman,gender: Woman,gender: Woman,...,gender: Woman,gender: Woman,gender: Man,gender: Man,gender: Man,gender: Man,gender: Man,gender: Woman,gender: Man,gender: Man
1,!Sample_characteristics_ch1,group: NW,group: OW/OB,group: MONW,group: OW/OB,group: NW,group: OW/OB,group: OW/OB,group: OW/OB,group: NW,...,group: OW/OB,group: OW/OB,group: OW/OB,group: NW,group: OW/OB,group: NW,group: OW/OB,group: OW/OB,group: OW/OB,group: NW
2,!Sample_characteristics_ch1,age: 21,age: 23,age: 10,age: 17,age: 11,age: 1,age: 18,age: 10,age: 12,...,age: 2,age: 2,age: 30,age: 17,age: 30,age: 19,age: 30,age: 2,age: 4,age: 19
3,!Sample_characteristics_ch1,fasting time: 6hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,...,fasting time: 6hr,fasting time: 4hr,fasting time: 4hr,fasting time: 6hr,fasting time: 6hr,fasting time: 4hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,fasting time: 6hr
4,!Sample_characteristics_ch1,timepoint: 0months,timepoint: 6months,timepoint: 0months,timepoint: 0months,timepoint: 0months,timepoint: 6months,timepoint: 6months,timepoint: 0months,timepoint: 0months,...,timepoint: 6months,timepoint: 0months,timepoint: 0months,timepoint: 0months,timepoint: 6months,timepoint: 0months,timepoint: 6months,timepoint: 0months,timepoint: 0months,timepoint: 0months


In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['gender: Man', 'gender: Woman'],
 1: ['group: NW', 'group: OW/OB', 'group: MONW'],
 2: ['age: 21',
  'age: 23',
  'age: 10',
  'age: 17',
  'age: 11',
  'age: 1',
  'age: 18',
  'age: 12',
  'age: 8',
  'age: 14',
  'age: 26',
  'age: 4',
  'age: 2',
  'age: 3',
  'age: 7',
  'age: 13',
  'age: 15',
  'age: 9',
  'age: 30',
  'age: 19'],
 3: ['fasting time: 6hr', 'fasting time: 4hr'],
 4: ['timepoint: 0months', 'timepoint: 6months']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'LDL Cholesterol Levels\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'LDL Cholesterol Levels\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is reco

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = True
trait_row_id = 1
age_row_id = 2
gender_row_id = 0

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    # Convert the value to lowercase for case-insensitive comparison
    lower_value = str(value).lower()

    # Check if the value represents the presence of LDL Cholesterol Levels
    if 'monw' in lower_value:
        return '1'
    else:
        # Absence of LDL Cholesterol Levels
        return '0'

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format


def convert_gender(value):
    # Convert the value to lowercase for case-insensitive comparison
    lower_value = str(value).lower()

    # Check if the value represents male or female
    if 'woman' in lower_value:
        return '1'
    else:
        # Return None for unknown values
        return '0'



In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM5494930,GSM5494931,GSM5494932,GSM5494933,GSM5494934,GSM5494935,GSM5494936,GSM5494937,GSM5494938,GSM5494939,...,GSM5494998,GSM5494999,GSM5495000,GSM5495001,GSM5495002,GSM5495003,GSM5495004,GSM5495005,GSM5495006,GSM5495007
LDL Cholesterol Levels,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,21,23,10,17,11,1,18,10,12,23,...,2,2,30,17,30,19,30,2,4,19
Gender,0,0,1,1,1,0,1,1,1,0,...,1,1,0,0,0,0,0,1,0,0


### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM5494930,GSM5494931,GSM5494932,GSM5494933,GSM5494934,GSM5494935,GSM5494936,GSM5494937,GSM5494938,...,GSM5494998,GSM5494999,GSM5495000,GSM5495001,GSM5495002,GSM5495003,GSM5495004,GSM5495005,GSM5495006,GSM5495007
0,7,5.505549,5.513184,4.986265,5.346870,5.531071,4.848168,5.063753,5.133687,6.971369,...,5.930267,5.864811,6.180203,6.302292,6.279264,6.346574,7.841532,6.446191,6.009474,5.820668
1,8,14.922929,14.996884,15.558723,15.464692,14.914828,14.921011,14.656432,15.027612,15.462935,...,14.720862,14.991377,15.062609,14.941717,15.565374,15.562799,15.232485,14.845854,14.843489,15.120225
2,15,10.185238,9.826678,9.003106,10.440502,8.448914,9.775237,9.587976,8.860986,8.913982,...,9.531411,9.137210,10.327923,9.672288,9.841389,9.210932,9.746890,8.812242,8.952330,8.907866
3,18,10.064262,10.235713,9.925906,9.915432,9.793022,9.425392,9.561722,9.583471,10.332012,...,9.351078,9.505807,9.616248,9.689660,9.452713,9.733175,9.733940,9.405109,9.631529,9.408484
4,20,6.339479,6.272158,6.426318,6.371126,6.117284,5.886479,5.469945,5.672563,7.095820,...,6.341808,6.790785,6.745134,7.003243,6.966419,6.912556,6.632631,6.584196,6.423572,6.826112


In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['7',
 '8',
 '15',
 '18',
 '20',
 '21',
 '24',
 '25',
 '29',
 '32',
 '39',
 '41',
 '42',
 '44',
 '45',
 '46',
 '48',
 '51',
 '53',
 '55']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['7', '8', '15', '18', '20', '21', '24', '25', '29', '32', '39', '41', '42', '44', '45', '46', '48', '51', '53', '55']\n"

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 'cad

In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 78 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'LDL Cholesterol Levels', the least common label is '1' with 6 occurrences. This represents 7.69% of the dataset.
The distribution of the feature 'LDL Cholesterol Levels' in this dataset is fine.

Quartiles for 'Age':
  25%: 8.0
  50% (Median): 11.0
  75%: 18.75
Min: 1
Max: 30
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 39 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [5]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE181339'

In [6]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
6,GSE181339,True,True,False,True,True,78.0,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [8]:
merged_data.head()

,LDL Cholesterol Levels,Age,Gender,A1BG,A1BG-AS1,A2M-AS1,A4GALT,AAAS,AACS,AADACL3,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0,21,0,9.356389,6.588705,10.086590,6.087023,8.855058,6.453003,6.644548,...,7.339372,6.278003,5.738853,5.108455,9.036174,7.664560,6.691724,10.559644,11.370012,8.635409
1,0,23,0,9.580217,6.861172,8.589913,5.958440,8.172307,6.070650,6.281522,...,7.143025,6.687131,6.101444,5.254791,9.805082,7.791538,6.815505,10.200364,11.444456,8.916923
2,1,10,1,9.920784,7.055549,9.467662,6.690681,8.768802,6.703954,7.138168,...,6.888677,6.648450,6.406971,5.426840,8.937783,7.575047,6.709371,11.124261,11.062703,8.439625
3,0,17,1,9.504974,6.792186,7.930585,5.814862,8.708854,6.803403,5.934699,...,7.263604,6.583358,6.144584,5.039489,9.203809,7.625925,6.840306,11.300428,10.987588,8.646433
4,0,11,1,9.533504,7.192053,9.596064,5.822462,8.534389,6.751145,6.999438,...,6.861367,6.475069,6.905870,5.460082,9.331111,8.048430,6.860686,10.561634,10.786569,8.603074


In [9]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [10]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [11]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [12]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [13]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 48.00% ± 11.47%


In [14]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 58.67% ± 7.77%


In [15]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [16]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0705
Found 49 genes with non-zero coefficients associated with the trait 'LDL Cholesterol Levels' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
   INO80C    -3.636229              3.636229
     TCP1     3.402646              3.402646
     POP1     3.046296              3.046296
  SNORA22     2.834675              2.834675
     IQCD    -2.762673              2.762673
      FTX     2.757990              2.757990
   CTAGE4     2.471257              2.471257
   CGREF1    -2.221289              2.221289
  SLC35B3    -1.972682              1.972682
 TMEM176B     1.779396              1.779396
KCND3-IT1     1.580360              1.580360
    DAAM1     1.326287              1.326287
    COPRS     1.119752              1.119752
     COQ6    -1.068776              1.068776
C20orf141     1.039900              1.03990

/content/drive/MyDrive/Colab Notebooks/utils.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [17]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0705
Found 2 significant genes associated with the trait 'LDL Cholesterol Levels' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
    DNLZ     0.216725 1.065003e-08           0.000063
  MRPL27    -0.198401 9.731653e-09           0.000063


/content/drive/MyDrive/Colab Notebooks/utils.py:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
